In [1]:
from dotenv import load_dotenv
_ = load_dotenv("config.env")

from common.mongodb import *
from common.files import *
from common.llms import *

doc = file2md("/Users/oliverkohn/repositories/datasphereAI/find_files/backend/data/receipt2.docx")

2025-01-31 18:50:07,403 - my_logger - INFO - host: localhost


host: localhost


In [5]:
document_types = ["Receipt", "Rental Contract", "Sales Agreement", "Non-Disclosure Agreement (NDA)", "Employment Contract", "Business Proposal", "Service Agreement", "Power of Attorney"]
document_types = ", ".join(document_types)

prompt_category= f"""Your task is to eassign below document to a category.
Possible categories are:

{document_types}

Answer in a JSON object of this format:

{{
    "category" : "xxx"
}}

Answer in a json in exactly this format. It is important that your answer only consists of this JSON and nothing else. No additional text.

This is the document you shall extract the information from:

{doc}
"""

llm = OllamaLLM(model="llama3.2:3b", temperature=0, cache=False)
category = execute_text_prompt(llm, prompt_category)
category

2025-01-31 18:50:54,616 - my_logger - INFO - Prompt Original:

Your task is to eassign below document to a category.
Possible categories are:

Receipt, Rental Contract, Sales Agreement, Non-Disclosure Agreement (NDA), Employment Contract, Business Proposal, Service Agreement, Power of Attorney

Answer in a JSON object of this format:

{
    "category" : "xxx"
}

Answer in a json in exactly this format. It is important that your answer only consists of this JSON and nothing else. No additional text.

This is the document you shall extract the information from:


# Official Receipt

Company Name: Tech Solutions UG

Address: Hauptstraße 123, 10115 Berlin, Germany

Phone: +49 30 12345678 | Email: contact@techsolutions.de

Website: www.techsolutions.de

Receipt Number: #20250131-002

Date of Issue: 31.01.2025

Customer Name: Max Mustermann

Customer Address: Musterstraße 45, 10117 Berlin, Germany

Customer Email: max.mustermann@email.com

Customer Phone: +49 170 9876543

| Item Description 

'{\n    "category" : "Receipt"\n}'

In [ ]:
prompt_general = f"""Your task is to extract this information from below document:

{{
    "receipt_id" : "xxx",
    "date" : "xxx",
    "items" : {{"price" : "xxx", "name" : "xxx"}}
    "total_price" : {{"amount": "xxx"}}
    "receipt_category" : "xxx"
}}

Answer in a json in exactly this format. It is important that your answer only consists of this JSON and nothing else. No additional text.

This is the document you shall extract the information from:

{doc}
"""

models = ["llama3.2:3b", "deepseek-r1:8b", "deepseek-r1:14b"]

r_llm = {}
for model in models:
    logger.info(f"RUN MODEL '{model}'")
    llm = OllamaLLM(model=model, temperature=0, cache=False)
    r_llm[model] = execute_text_prompt(llm, prompt_general)

In [ ]:
j_llm = {}
for k, v in r_llm.items():
    j_llm[k] = response2json(v)

j_llm

In [ ]:
prompt_items = f"""Below you find a receipt. Your task is to identify the items from this receipt and extract their names as well as their price.
Answer in a json object like this:

{{
    "response" : [{{"item_name" : "xxx", "item_price" : "xxx"}}]
}}

It is important that you answer exactly in this format, a JSON object where the main key is "response". Do not put anything els ein your response.

This is the document you shall extract the information from:

{doc}
"""

models = ["llama3.2:3b", "deepseek-r1:8b", "deepseek-r1:14b"]

r_llm_specific = {}
for model in models:
    logger.info(f"RUN MODEL '{model}'")
    llm = OllamaLLM(model=model, temperature=0, cache=False)
    r_llm_specific[model] = execute_text_prompt(llm, prompt_items)


In [ ]:
j_llm = {}
for k, v in r_llm_specific.items():
    j_llm[k] = response2json(v)
    print(v)

j_llm